In [5]:
!pip install -q kaggle

In [6]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"taban123456","key":"0e301cbc4b47db5ed837878cdfc1c469"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

gender-recognition-200k-images-celeba.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!unzip gender-recognition-200k-images-celeba

Archive:  gender-recognition-200k-images-celeba.zip
replace Dataset/Test/Female/160001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [10]:
!pip install wandb

In [11]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras import models ,layers, optimizers
from  tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
import wandb
from wandb.keras import WandbCallback
import seaborn as sn

In [12]:
!pwd

/content


# Hiper_Parameters

In [13]:
batch_size = 256
epochs = 10
lr = 0.001
width = height = 64

# Dataset

In [14]:
data_generator = ImageDataGenerator(rescale = 1./255,
                                    horizontal_flip = True)


In [15]:
train_data = data_generator.flow_from_directory('Dataset/Train',
                                                batch_size = batch_size,
                                                shuffle = True,
                                                class_mode = 'binary',
                                                target_size = (width , height) )

Found 160000 images belonging to 2 classes.


In [16]:
val_data = data_generator.flow_from_directory('Dataset/Validation',
                                                batch_size = batch_size,
                                                shuffle = True,
                                                class_mode = 'binary',
                                                target_size = (width , height) )

Found 22598 images belonging to 2 classes.


In [17]:
model = tf.keras.models.Sequential([
                                    layers.Conv2D(96,(11,11), activation = 'relu', input_shape = (width, height,3)),
                                    layers.BatchNormalization(),
                                    layers.MaxPooling2D(2),
                                    layers.Conv2D(256,(11,11), activation = 'relu', input_shape = (width, height,3)),
                                    layers.BatchNormalization(),
                                    layers.Conv2D(384,(3,3), activation = 'relu', input_shape = (width, height,3)),
                                    layers.BatchNormalization(),
                                    layers.Conv2D(384,(3,3), activation = 'relu', input_shape = (width, height,3)),
                                    layers.BatchNormalization(),
                                    layers.Conv2D(256,(3,3), activation = 'relu', input_shape = (width, height,3)),
                                    layers.BatchNormalization(),
                                    layers.MaxPooling2D(2, strides = (2,2)),
                                    layers.Flatten(),
                                    layers.Dense(4096, activation='relu'),
                                    layers.Dropout(0.5),
                                    layers.Dense(4096,activation='relu'),
                                    layers.Dropout(0.5),
                                    layers.Dense(1,activation='sigmoid')

])

In [18]:
!pip  install wandb

In [19]:
wandb.init(project="gender_recognition")

config = wandb.config
config.learning_rate = 0.001

wandb: Currently logged in as: cyber (use `wandb login --relogin` to force relogin)


In [20]:
from tensorflow.keras.optimizers import  RMSprop,Adam,SGD

In [21]:
model.compile(optimizer=RMSprop(learning_rate = config.learning_rate),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [22]:
model.fit(train_data,
          validation_data = val_data,
          epochs = epochs)

Epoch 1/10
625/625 [==============================] - 364s 535ms/step - loss: 0.8831 - accuracy: 0.7981 - val_loss: 0.1789 - val_accuracy: 0.9275
Epoch 2/10
625/625 [==============================] - 329s 527ms/step - loss: 0.1911 - accuracy: 0.9321 - val_loss: 0.5141 - val_accuracy: 0.9253
Epoch 3/10
625/625 [==============================] - 327s 523ms/step - loss: 0.2208 - accuracy: 0.9468 - val_loss: 0.1599 - val_accuracy: 0.9430
Epoch 4/10
625/625 [==============================] - 328s 525ms/step - loss: 0.1231 - accuracy: 0.9551 - val_loss: 0.2852 - val_accuracy: 0.8507
Epoch 5/10
625/625 [==============================] - 328s 525ms/step - loss: 0.1153 - accuracy: 0.9605 - val_loss: 0.2926 - val_accuracy: 0.8913
Epoch 6/10
625/625 [==============================] - 329s 526ms/step - loss: 0.1283 - accuracy: 0.9622 - val_loss: 0.2615 - val_accuracy: 0.9412
Epoch 7/10
625/625 [==============================] - 330s 528ms/step - loss: 0.0997 - accuracy: 0.9654 - val_loss: 0.4346 -

In [23]:
model.save('Gender_detection.h5')

In [24]:
test_data = data_generator.flow_from_directory('Dataset/Test',
                                                batch_size = batch_size,
                                                shuffle = True,
                                                class_mode = 'binary',
                                                target_size = (width , height) )

Found 20001 images belonging to 2 classes.


In [25]:
model.evaluate(test_data)

79/79 [==============================] - 24s 303ms/step - loss: 0.1700 - accuracy: 0.9667


[0.17001238465309143, 0.9667016863822937]

In [26]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[11542     0]
 [ 8459     0]]
Classification Report
              precision    recall  f1-score   support

      Female       0.58      1.00      0.73     11542
        Male       0.00      0.00      0.00      8459

    accuracy                           0.58     20001
   macro avg       0.29      0.50      0.37     20001
weighted avg       0.33      0.58      0.42     20001



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
cm = confusion_matrix(val_data.classes,y_pred)
plt.figure(figsize=(12,10))
sn.heatmap(cm, annot=True)

NameError: ignored